# DataStax Astra DB

In [5]:
import os
from dotenv import load_dotenv

# Import DB libaries
from langchain_astradb import AstraDBVectorStore
from langchain_groq import ChatGroq
from langchain_openai import OpenAIEmbeddings

# Import langchain core and community libraries
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader, PyPDFLoader, DirectoryLoader

In [6]:
load_dotenv()

True

In [7]:
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")
os.environ['ASTRA_DB_APPLICATION_TOKEN'] = os.getenv("ASTRA_DB_APPLICATION_TOKEN")
os.environ['ASTRA_DB_API_ENDPOINT'] = os.getenv("ASTRA_DB_API_ENDPOINT")
os.environ['ASTRA_DB_KEYSPACE'] = os.getenv("ASTRA_DB_KEYSPACE")
os.environ['GROQ_API_KEY'] = os.getenv("GROQ_API_KEY")

In [8]:
# OpenAI Embeddings model
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
embeddings

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x14e1ec7d0>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x14e1ec890>, model='text-embedding-3-small', dimensions=None, deployment='text-embedding-ada-002', openai_api_version=None, openai_api_base=None, openai_api_type=None, openai_proxy=None, embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

In [ ]:
# Load documents from the data directory
loader=DirectoryLoader("./data/", glob="*.txt", loader_cls=TextLoader)
documents = loader.load()
print(f"Loaded {len(documents)} documents.")

Loaded 3 documents.


In [10]:
# Split documents into chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500, 
    chunk_overlap=50, 
    length_function=len,
    separators=[' ']
    )
chunks = text_splitter.split_documents(documents)
print(f"Split into {len(chunks)} chunks.")

Split into 5 chunks.


In [14]:
# Create AstraDB Vector store
vector_store = AstraDBVectorStore(
    collection_name="rag_udemy",
    embedding=embeddings,
    api_endpoint=os.getenv("ASTRA_DB_API_ENDPOINT"),
    token=os.getenv("ASTRA_DB_APPLICATION_TOKEN"),
    namespace=os.getenv("ASTRA_DB_KEYSPACE"),
)
print(vector_store)
vector_store.add_documents(chunks)
print(f"Documents with {len(chunks)} chunks added to AstraDB vector store.")

Documents with 5 chunks added to AstraDB vector store.


In [15]:
# Create retriever
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={'k':3})
retriever

VectorStoreRetriever(tags=['AstraDBVectorStore', 'OpenAIEmbeddings'], vectorstore=<langchain_astradb.vectorstores.AstraDBVectorStore object at 0x14e1e5410>, search_kwargs={'k': 3})

In [16]:
# Initialize Groq LLM
llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0,
    max_tokens=1000,
)
llm

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 32768, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x14f3aced0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x1601428d0>, model_name='llama-3.3-70b-versatile', temperature=1e-08, model_kwargs={}, groq_api_key=SecretStr('**********'), max_tokens=1000)

In [17]:
# Create RAG Prompt Template
prompt_template = """You are a helpful AI assistant. Answer the question based on the provided context.
If you cannot find the answer in the context, simply state that you do not have enough information. 
Do not make up an answer.

Context: 
{context}

Question: {question}

Answer:"""
prompt = ChatPromptTemplate.from_template(prompt_template)
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='You are a helpful AI assistant. Answer the question based on the provided context.\nIf you cannot find the answer in the context, simply state that you do not have enough information. \nDo not make up an answer.\n\nContext: \n{context}\n\nQuestion: {question}\n\nAnswer:'), additional_kwargs={})])

In [18]:
# Build RAG Chain
def format_context(docs) -> str:
    """ Format retrieved documents into a single context string."""
    return "\n\n".join([doc.page_content for doc in docs])

rag_chain = (
    {
        "context": retriever | format_context,
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()
)
rag_chain

{
  context: VectorStoreRetriever(tags=['AstraDBVectorStore', 'OpenAIEmbeddings'], vectorstore=<langchain_astradb.vectorstores.AstraDBVectorStore object at 0x14e1e5410>, search_kwargs={'k': 3})
           | RunnableLambda(format_context),
  question: RunnablePassthrough()
}
| ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='You are a helpful AI assistant. Answer the question based on the provided context.\nIf you cannot find the answer in the context, simply state that you do not have enough information. \nDo not make up an answer.\n\nContext: \n{context}\n\nQuestion: {question}\n\nAnswer:'), additional_kwargs={})])
| ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 32768, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_out

In [ ]:
# Test RAG Chain with a query
query = "What is deep learning?"
response = rag_chain.invoke(query)
print(f"Question: {query}")
print(f"Response: {response}")

Question: What is deep learning?
Response: Deep learning is a subset of machine learning based on artificial neural networks, which are inspired by the human brain and consist of layers of interconnected nodes.


In [22]:
# Test RAG Chain with a query list
queries = [
    "What is deep learning?",
    "What is machine learning?",
    "What are the types of machine learning?"
]

# Get contexts and responses
contexts  = retriever.batch(queries)
responses = rag_chain.batch(queries)

# Display results
results = {
    query: {"contexts": context, "response": response}
    for query, context, response in zip(queries, contexts, responses)
}

for query, data in results.items():
    print("=" * 60)
    print(f"Question: {query}\n")
    print(f"\nRetrieved {len(data['contexts'])} documents")
    for i, doc in enumerate(data['contexts'], 1):
        print(f"\nDocument {i}: {doc.page_content[:100]}...")
    print(f"\nResponse: {data['response']}\n")

Question: What is deep learning?


Retrieved 3 documents

Document 1: Deep Learning and Neural Networks

    Deep learning is a subset of machine learning based on artifi...

Document 2: Machine Learning Fundamentals

    Machine learning is a subset of artificial intelligence that enab...

Document 3: Natural Language Processing (NLP)

    NLP is a field of AI that focuses on the interaction between ...

Response: Deep learning is a subset of machine learning based on artificial neural networks, which are inspired by the human brain and consist of layers of interconnected nodes.

Question: What is machine learning?


Retrieved 3 documents

Document 1: Machine Learning Fundamentals

    Machine learning is a subset of artificial intelligence that enab...

Document 2: Deep Learning and Neural Networks

    Deep learning is a subset of machine learning based on artifi...

Document 3: Natural Language Processing (NLP)

    NLP is a field of AI that focuses on the interaction between ...



In [23]:
# Similarity search
search_results=vector_store.similarity_search("What is artificial intelligence?", k=3)  
print("Similarity Search Results:\n")
for i, doc in enumerate(search_results, 1):
    print(f"Result {i}: {doc.page_content}\n")

search_results_with_scores=vector_store.similarity_search_with_score("What is artificial intelligence?", k=3)  
print("Similarity Search Results with Scores:\n")
for i, (doc, score) in enumerate(search_results_with_scores, 1):
    print(f"Result {i}:\t Score: {score}\n {doc.page_content}\n")

Similarity Search Results:

Result 1: Machine Learning Fundamentals

    Machine learning is a subset of artificial intelligence that enables systems to learn 
    and improve from experience without being explicitly programmed. There are three main 
    types of machine learning: supervised learning, unsupervised learning, and reinforcement 
    learning. Supervised learning uses labeled data to train models, while unsupervised 
    learning finds patterns in unlabeled data. Reinforcement learning learns through

Result 2: Natural Language Processing (NLP)

    NLP is a field of AI that focuses on the interaction between computers and human language. 
    Key tasks in NLP include text classification, named entity recognition, sentiment analysis, 
    machine translation, and question answering. Modern NLP heavily relies on transformer 
    architectures like BERT, GPT, and T5. These models use attention mechanisms to understand 
    context and relationships between words in text.

Re